<a href="https://colab.research.google.com/github/hikmahealth/covid19countymap/blob/master/notebooks/County_Prioritization_Filler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import io
import copy
import urllib

from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread
import pandas as pd

Download county populations from census.gov

In [0]:
with urllib.request.urlopen("https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv") as infile:
  raw_data = infile.read()
# Deal with encoding issues in the Census CSV file.
replacements = {0xed: "í", 0xe1: "á", 0xf3: "ó", 0xf1: "ñ", 0xfc:"ü"}
for char, repl in replacements.items():
  raw_data = raw_data.replace(bytes([char]), repl.encode())
raw_census = pd.read_csv(io.BytesIO(raw_data))

In [0]:
census = raw_census[raw_census.COUNTY != 0].copy()

In [0]:
census["fips"] = census.STATE * 1000 + census.COUNTY
census.set_index("fips", inplace=True)
census.sort_values(by="POPESTIMATE2019", inplace=True, ascending=False)

Prepare the Google sheet we will write the prioritzation to.

In [0]:
auth.authenticate_user()
sheets_client = gspread.authorize(GoogleCredentials.get_application_default())
sheet = sheets_client.open_by_url("https://docs.google.com/spreadsheets/d/1bu7sY3R4AQagFietD0hMmGAFo_W0-PpbcfRLcADtc3s/edit#gid=0").sheet1

In [0]:
sheet.resize(rows=(len(census) // 50 + 1) * 50 + 1, cols=5)

In [0]:
header = sheet.range('A1:D1')
header[0].value = "FIPS"
header[1].value = "County"
header[2].value = "State"
header[3].value = "Link"
sheet.update_cells(header);

In [0]:
_FORM = "https://docs.google.com/forms/d/e/1FAIpQLScO4B6PkJsOeVKM2_dbpFMhnXCfb89Kya9bWtKX4uUWIMev7Q/viewform?usp=pp_url&entry.1112165839="

In [0]:
cells = None
for i, (fips, row) in enumerate(census.iterrows()):
  # Update the sheet in batches of 50
  if not i % 50:
    if cells:
      sheet.update_cells(cells)
    cells = sheet.range("A%d:D%d" % (i + 2, i + 52))
  row_index = (i % 50) * 4
  cells[row_index].value = fips
  cells[row_index + 1].value = row.CTYNAME
  cells[row_index + 2].value = row.STNAME
  full_name = row.CTYNAME + ", " + row.STNAME
  cells[row_index + 3].value = _FORM + urllib.parse.quote(full_name)
sheet.update_cells(cells);